<a href="https://colab.research.google.com/github/jackhu1105/tku-AI-412631599/blob/main/%E8%BB%8A%E7%89%8C%E8%BE%A8%E8%AD%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2 as cv
import numpy as np

src = cv.imread("D:/license/license1.jpg")

def license_preparation(image):
    image_hsv = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    low_hsv = np.array([0, 0, 221])
    high_hsv = np.array([180, 30, 255])
    mask = cv.inRange(image_hsv, low_hsv, high_hsv)
    cv.imshow('mask', mask)
    image_dst = cv.bitwise_and(image, image, mask=mask)
    cv.imshow('license_dst', image_dst)
    image_blur = cv.GaussianBlur(image_dst, (7, 7), 0)  # Fixed the GaussianBlur call
    cv.imshow('license_blur', image_blur)
    image_gray = cv.cvtColor(image_blur, cv.COLOR_BGR2GRAY)
    ret, binary = cv.threshold(image_gray, 0, 255, cv.THRESH_BINARY | cv.THRESH_OTSU)
    cv.imshow('binary', binary)
    kernel1 = cv.getStructuringElement(cv.MORPH_RECT, (4, 6))
    image_open = cv.morphologyEx(binary, cv.MORPH_OPEN, kernel1)
    cv.imshow('license_open', image_open)
    kernel2 = cv.getStructuringElement(cv.MORPH_RECT, (7, 7))
    image_closed = cv.morphologyEx(image_open, cv.MORPH_CLOSE, kernel2)
    cv.imshow('license_closed', image_closed)  # Removed the '=' sign here
    cv.namedWindow('output', 0)
    return image_closed

license_prepared = license_preparation(src)

contours, hierarchy = cv.findContours(license_prepared, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

def choose_license_area(contours, Min_Area):
    temp_contours = []
    for contour in contours:
        if cv.contourArea(contour) > Min_Area:
            temp_contours.append(contour)
    license_area = []
    for temp_contour in temp_contours:
        rect_tupple = cv.minAreaRect(temp_contour)
        rect_width, rect_height = rect_tupple[1]
        if rect_width < rect_height:
            rect_width, rect_height = rect_height, rect_width
        aspect_ratio = rect_width / rect_height
        if 2 < aspect_ratio < 5.5:
            license_area.append(temp_contour)
    return license_area

license_area = choose_license_area(contours, 1000)  # Corrected the second argument

def license_segment(license_area):
    if len(license_area) == 1:
        for car_plate in license_area:
            x, y, w, h = cv.boundingRect(car_plate)
            card_img = src[y:y+h, x:x+w]
            cv.imwrite("card_img.jpg", card_img)  # Added file extension
            return card_img

result = license_segment(license_area)
cv.imshow('result', result)

# 添加适当的窗口事件循环和关闭窗口的代码
cv.waitKey(0)
cv.destroyAllWindows()


In [ ]:
pip install opencv-python
